<a href="https://colab.research.google.com/github/sahug/ds-bert/blob/main/BERT%20NLP%20-%20Masked%20Language%20Modeling%20using%20BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**BERT NLP - Casual Language Modeling using BERT**

**Language modeling** predicts words in a sentence. There are two forms of language modeling.
- **Causal Language Modeling** predicts the next token in a sequence of tokens, and the model can only attend to tokens on the left. `distilgpt2`
- **Masked Language Modeling** predicts a masked token in a sequence, and the model can attend to tokens bidirectionally. `distilroberta-base`






In [1]:
%pip install -qq datasets

     |████████████████████████████████| 346 kB 5.0 MB/s 
     |████████████████████████████████| 1.1 MB 20.7 MB/s 
     |████████████████████████████████| 86 kB 3.0 MB/s 
     |████████████████████████████████| 86 kB 2.4 MB/s 
     |████████████████████████████████| 212 kB 48.7 MB/s 
     |████████████████████████████████| 140 kB 12.3 MB/s 
     |████████████████████████████████| 596 kB 39.1 MB/s 
     |████████████████████████████████| 127 kB 15.1 MB/s 
     |████████████████████████████████| 94 kB 856 kB/s 
     |████████████████████████████████| 144 kB 38.3 MB/s 
     |████████████████████████████████| 271 kB 48.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


**Load Dataset**

In [2]:
from datasets import load_dataset
eli5 = load_dataset("eli5", split="train_asks[:5000]")

Downloading:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/286M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/36.2M [00:00<?, ?B/s]

Dataset eli5 downloaded and prepared to /root/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa. Subsequent calls will reuse this data.


**Train and Test Split**

In [3]:
eli5 = eli5.train_test_split(test_size=0.2)

In [4]:
eli5

DatasetDict({
    train: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['q_id', 'title', 'selftext', 'document', 'subreddit', 'answers', 'title_urls', 'selftext_urls', 'answers_urls'],
        num_rows: 1000
    })
})

In [5]:
eli5["train"][0], eli5["test"][0] 

({'answers': {'a_id': ['c51x5y6', 'c51wtax'],
   'score': [2, 2],
   'text': ["It's a fair question: We can't derive them very well, actually. \n\nWith fluids, you traditionally model it at the macroscopic end, where it's assumed to be homogeneous if not more (say, incompressible and whatnot). You use parameters describing bulk properties such as density, dynamic and kinematic viscosity and so on. Physically/conceptually the models are quite simple, although the resulting equations are not. ([Navier-Stokes](_URL_3_), for instance)\n\nOn the other end, you have physical chemistry and the basic interactions of molecules. ([intermolecular forces](_URL_2_)) These do depend on the shapes of the molecules, but a lot more than just that. Generally, all molecules attract at far ranges and repel at short range (although the distances and the amount of attraction vary quite a bit). \n\nEven that is a bit simplified, since you're not taking into account chemistry/chemical reactions, which can be 

**Look at Dataset**

In [6]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [8]:
show_random_elements(eli5["train"])

**Extract Text**

**Flatten** the dataset for easy extraction. We will be able to extract the data like `answers.text` instead of `["answers"]["text"]`

In [9]:
eli5 = eli5.flatten()

In [10]:
eli5["train"]["answers.text"][0]

["It's a fair question: We can't derive them very well, actually. \n\nWith fluids, you traditionally model it at the macroscopic end, where it's assumed to be homogeneous if not more (say, incompressible and whatnot). You use parameters describing bulk properties such as density, dynamic and kinematic viscosity and so on. Physically/conceptually the models are quite simple, although the resulting equations are not. ([Navier-Stokes](_URL_3_), for instance)\n\nOn the other end, you have physical chemistry and the basic interactions of molecules. ([intermolecular forces](_URL_2_)) These do depend on the shapes of the molecules, but a lot more than just that. Generally, all molecules attract at far ranges and repel at short range (although the distances and the amount of attraction vary quite a bit). \n\nEven that is a bit simplified, since you're not taking into account chemistry/chemical reactions, which can be important even to a fluid that doesn't seem to be 'reacting'. For instance, w

**Preprocess**

In [11]:
%pip install -qq transformers

     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 43.8 MB/s 


In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [13]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]], truncation=True)

In [14]:
tokenized_eli5 = eli5.map(preprocess_function, batched=True, num_proc=4, remove_columns=eli5["train"].column_names)

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

In [15]:
tokenized_eli5

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [16]:
print(tokenized_eli5["train"]["input_ids"][0])

[0, 243, 18, 10, 2105, 864, 35, 166, 64, 75, 34882, 106, 182, 157, 6, 888, 4, 1437, 50118, 50118, 3908, 26572, 6, 47, 10341, 1421, 24, 23, 5, 12303, 3866, 18137, 253, 6, 147, 24, 18, 9159, 7, 28, 9486, 42019, 114, 45, 55, 36, 28357, 6, 45059, 5224, 4748, 8, 99, 3654, 322, 370, 304, 17294, 9072, 8533, 3611, 215, 25, 16522, 6, 6878, 8, 449, 10748, 5183, 17737, 16254, 1571, 8, 98, 15, 4, 27592, 3435, 73, 38498, 13851, 5, 3092, 32, 1341, 2007, 6, 1712, 5, 5203, 43123, 32, 45, 4, 47567, 487, 37041, 12, 5320, 6568, 47620, 1215, 42703, 1215, 246, 1215, 238, 13, 4327, 43, 50118, 50118, 4148, 5, 97, 253, 6, 47, 33, 2166, 11877, 8, 5, 3280, 11324, 9, 20237, 4, 47567, 8007, 119, 4104, 32188, 1572, 47620, 1215, 42703, 1215, 176, 1215, 35122, 1216, 109, 6723, 15, 5, 16499, 9, 5, 20237, 6, 53, 10, 319, 55, 87, 95, 14, 4, 25817, 6, 70, 20237, 5696, 23, 444, 16296, 8, 2851, 523, 23, 765, 1186, 36, 24648, 5, 21459, 8, 5, 1280, 9, 13003, 10104, 1341, 10, 828, 322, 1437, 50118, 50118, 8170, 14, 16, 10, 8

**Capture Truncated Text**
When we tokenize texts the tokenizer truncates some of the texts based on default size. So we need a second preprocessing function to capture text truncated from any lengthy examples to prevent loss of information. This preprocessing function should:

- Concatenate all the text.
- Split the concatenated text into smaller chunks defined by block_size.

In [17]:
BLOCK_SIZE = 128

def group_text(examples):
  concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
  total_length = len(concatenated_examples[list(examples.keys())[0]])
  result = {
      k: [t[i: i+ BLOCK_SIZE] for i in range(0, total_length, BLOCK_SIZE)]
      for k, t in concatenated_examples.items()
      }
  result["labels"] = result["input_ids"].copy()
  return result

In [18]:
lm_dataset = tokenized_eli5.map(group_text, batched=True, num_proc=4)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [19]:
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7420
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1909
    })
})

In [20]:
print(lm_dataset["train"]["input_ids"][0])

[0, 243, 18, 10, 2105, 864, 35, 166, 64, 75, 34882, 106, 182, 157, 6, 888, 4, 1437, 50118, 50118, 3908, 26572, 6, 47, 10341, 1421, 24, 23, 5, 12303, 3866, 18137, 253, 6, 147, 24, 18, 9159, 7, 28, 9486, 42019, 114, 45, 55, 36, 28357, 6, 45059, 5224, 4748, 8, 99, 3654, 322, 370, 304, 17294, 9072, 8533, 3611, 215, 25, 16522, 6, 6878, 8, 449, 10748, 5183, 17737, 16254, 1571, 8, 98, 15, 4, 27592, 3435, 73, 38498, 13851, 5, 3092, 32, 1341, 2007, 6, 1712, 5, 5203, 43123, 32, 45, 4, 47567, 487, 37041, 12, 5320, 6568, 47620, 1215, 42703, 1215, 246, 1215, 238, 13, 4327, 43, 50118, 50118, 4148, 5, 97, 253, 6, 47, 33, 2166, 11877, 8, 5, 3280, 11324, 9, 20237]


In [21]:
print(lm_dataset["train"]["labels"][0])

[0, 243, 18, 10, 2105, 864, 35, 166, 64, 75, 34882, 106, 182, 157, 6, 888, 4, 1437, 50118, 50118, 3908, 26572, 6, 47, 10341, 1421, 24, 23, 5, 12303, 3866, 18137, 253, 6, 147, 24, 18, 9159, 7, 28, 9486, 42019, 114, 45, 55, 36, 28357, 6, 45059, 5224, 4748, 8, 99, 3654, 322, 370, 304, 17294, 9072, 8533, 3611, 215, 25, 16522, 6, 6878, 8, 449, 10748, 5183, 17737, 16254, 1571, 8, 98, 15, 4, 27592, 3435, 73, 38498, 13851, 5, 3092, 32, 1341, 2007, 6, 1712, 5, 5203, 43123, 32, 45, 4, 47567, 487, 37041, 12, 5320, 6568, 47620, 1215, 42703, 1215, 246, 1215, 238, 13, 4327, 43, 50118, 50118, 4148, 5, 97, 253, 6, 47, 33, 2166, 11877, 8, 5, 3280, 11324, 9, 20237]


For **Causal Language Modeling**, use `DataCollatorForLanguageModeling` to create a batch of examples. It will also dynamically pad your text to the length of the longest element in its batch, so they are a uniform length. While it is possible to pad your text in the tokenizer function by setting padding=True, dynamic padding is more efficient.

In [22]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False, return_tensors="tf")

**Train**

To **fine-tune** a model in TensorFlow, start by converting your datasets to the tf.data.Dataset format with to_tf_dataset. Specify inputs and labels in columns, whether to shuffle the dataset order, batch size, and the data collator:

In [23]:
tf_train_set = lm_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    dummy_labels=True,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = lm_dataset["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    dummy_labels=True,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

**Optimizer**

In [24]:
from transformers import create_optimizer, AdamWeightDecay
optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

**Model**

In [25]:
from transformers import TFAutoModelForCausalLM
model = TFAutoModelForCausalLM.from_pretrained("distilroberta-base")

Downloading:   0%|          | 0.00/465M [00:00<?, ?B/s]

If you want to use `TFRobertaLMHeadModel` as a standalone, add `is_decoder=True.`
All model checkpoint layers were used when initializing TFRobertaForCausalLM.

All the layers of TFRobertaForCausalLM were initialized from the model checkpoint at distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForCausalLM for predictions without further training.


**Compile**

In [26]:
import tensorflow as tf
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


**Fit**

In [ ]:
model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)

Epoch 1/3
  4/463 [..............................] - ETA: 2:55:16 - loss: 16.3984